# Análisis Exploratorio:
Describir la serie de tiempo y visualizarla.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def mostrar_graficas(archivos):
    for archivo in archivos:
        # Leemos los datos
        datos = pd.read_csv(archivo + ".csv", parse_dates=True, index_col=0)
        
        # Hacemos el dibujo
        plt.figure(figsize=(12, 6))
        datos.plot(ax=plt.gca())  # gca es como decir "dame el eje actual para dibujar"
        plt.title(archivo)
        plt.ylabel(datos.columns[0])
        plt.xlabel("Día o Mes")
        plt.legend()
        plt.grid(True)
        plt.show()

# Aquí van los nombres de los archivos que vamos a graficar
nombres_archivos = [
    "daily-total-female-births",
    "shampoo",
    "monthly-mean-temp",
    "monthly-car-sales"
]

mostrar_graficas(nombres_archivos)

# Evaluacion de modelo
- MSE 
- RMSE

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

def evaluate_models(y_true, y_pred):
    # RMSE
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    # MAE
    mae = mean_absolute_error(y_true, y_pred)
    
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")


def get_y_pred_y_(data, y_pred_column, y_true_column):
    data_clean = data.dropna(subset=[y_pred_column, y_true_column])
    
    y_pred = data_clean[y_pred_column]
    y_true = data_clean[y_true_column]
    
    return y_true, y_pred


# Promedios:
• Aplicar métodos de promedios y comparar los resultados con el conjunto original.

In [ ]:
def graficar_con_promedio(filename, ventana=3):
    datos = pd.read_csv(filename, parse_dates=True, index_col=0)
    
    # Sacamos el promedio de los últimos datos y lo ponemos un poquito adelante para "adivinar" el siguiente dato
    datos['Adivinado'] = datos.rolling(ventana).mean().shift(-ventana+1)
    
    # Hacemos el dibujito
    plt.figure(figsize=(12, 6))
    plt.plot(datos.index, datos[datos.columns[0]], label='Real')
    plt.plot(datos.index, datos['Adivinado'], label='Adivinado', color='red')
    
    plt.title(filename)
    plt.ylabel(datos.columns[0])
    plt.xlabel("Día o Mes")
    plt.legend()
    plt.grid(True)
    plt.show()
    return datos

# Estos son los datos con los que vamos a trabajar
archivos = [
    "daily-total-female-births.csv",
    "shampoo.csv",
    "monthly-mean-temp.csv",
    "monthly-car-sales.csv"
]

# Cuántos datos tomamos para el promedio
ventana = 3

for archivo in archivos:
    datos = graficar_con_promedio(archivo, ventana)
    y_true, y_pred  = get_y_pred_y_(data=datos, y_pred_column="Adivinado", y_true_column=datos.columns[0])
    evaluate_models(y_true, y_pred)





# 3. Sarima

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

def predecir_con_sarima(nombre_archivo, numero_de_predicciones=12):
    # Leer el archivo
    datos = pd.read_csv(nombre_archivo + ".csv")
    
    # Tomar solo la columna de valores
    valores = datos[datos.columns[1]].values
    
    # Crear el modelo y entrenarlo
    modelo = SARIMAX(valores, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    modelo_listo = modelo.fit(disp=False)
    
    # Hacer las predicciones para fechas futuras
    predicciones = modelo_listo.predict(len(valores), len(valores) + numero_de_predicciones - 1)
    
    # Hacer predicciones dentro del rango de tiempo existente (insample)
    pred_sarima_insample = modelo_listo.predict(0, len(valores) - 1)
    
    # Evaluar el rendimiento del modelo con datos existentes
    evaluate_models(valores, pred_sarima_insample)
    
    # Mostrar los resultados
    plt.figure(figsize=(12, 6))
    plt.plot(valores, label='Datos reales')
    plt.plot(range(len(valores), len(valores) + numero_de_predicciones), predicciones, color='red', label='Predicciones')
    plt.title(nombre_archivo)
    plt.legend()
    plt.show()

    return valores, predicciones

# Archivos a analizar
archivos = [
    "daily-total-female-births",
    "shampoo",
    "monthly-mean-temp",
    "monthly-car-sales"
]

# Para cada archivo, hacer la predicción y mostrarla
for archivo in archivos:
    y_true, y_pred = predecir_con_sarima(archivo)

# 4. Alisamiento exponencial

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX

def evaluate_models_2(y_true, y_pred, model_name):
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f'{model_name} - RMSE: {rmse}, MAE: {mae}')
    
def predice_cosas(archivo, pasos=12):
    # Abrir archivo
    datos = pd.read_csv(archivo + ".csv")
    
    # Tomar solo la columna con los números
    serie = datos[datos.columns[1]].values
    
    # Alisado simple
    modelo_simple = SimpleExpSmoothing(serie).fit()
    pred_simple = modelo_simple.forecast(steps=pasos)
    pred_simple_insample = modelo_simple.fittedvalues
    evaluate_models_2(serie, pred_simple_insample, "Alisado simple")
    
    # Alisado doble
    modelo_doble = ExponentialSmoothing(serie, trend='add').fit()
    pred_doble = modelo_doble.forecast(steps=pasos)
    pred_doble_insample = modelo_doble.fittedvalues
    evaluate_models_2(serie, pred_doble_insample, "Alisado doble")
    
    # SARIMA (modelo complicado)
    modelo_sarima = SARIMAX(serie, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    sarima_listo = modelo_sarima.fit(disp=False)
    pred_sarima = sarima_listo.predict(len(serie), len(serie) + pasos - 1)
    pred_sarima_insample = sarima_listo.predict(0, len(serie) - 1)
    evaluate_models_2(serie, pred_sarima_insample, "SARIMA")
    
    # Hacer dibujito
    plt.figure(figsize=(12, 6))
    plt.plot(serie, label='Datos de verdad')
    plt.plot(range(len(serie), len(serie) + pasos), pred_simple, label='Alisado simple', color='blue')
    plt.plot(range(len(serie), len(serie) + pasos), pred_doble, label='Alisado doble', color='green')
    plt.plot(range(len(serie), len(serie) + pasos), pred_sarima, label='El modelo complicado', color='red')
    plt.title(archivo)
    plt.legend()
    plt.show()

# Archivos para mirar
mis_archivos = [
    "daily-total-female-births",
    "shampoo",
    "monthly-mean-temp",
    "monthly-car-sales"
]

# Hacer todo por cada archivo
for archivo in mis_archivos:
    predice_cosas(archivo)



# 5. Prophet

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet

def convertir_fecha_custom(fecha):
    year, month = fecha.split('-')
    year_base = 2021
    year_final = year_base + int(year) - 1
    return pd.to_datetime(f"{year_final}-{month}")

def convertir_fecha(fecha, archivo):
    if archivo == "shampoo":
        return convertir_fecha_custom(fecha)
    else:
        formatos = {
            "daily-total-female-births": "%Y-%m-%d",
            "monthly-mean-temp": "%Y-%m",
            "monthly-car-sales": "%Y-%m"
        }
        return pd.to_datetime(fecha, format=formatos[archivo], errors='coerce')

def predice_con_prophet(archivo, pasos=12):
    # Abrir archivo, suponiendo que la primera fila es el encabezado
    datos = pd.read_csv(archivo + ".csv", header=0)
    
    # Convertir la fecha a formato datetime basado en el archivo
    datos['ds'] = datos[datos.columns[0]].apply(lambda x: convertir_fecha(x, archivo))
    
    # Verificar y eliminar filas con fechas NaN
    filas_con_nan = datos[datos['ds'].isna()]
    if not filas_con_nan.empty:
        print(f"Eliminando filas con fechas no válidas en {archivo}:")
        print(filas_con_nan)
        datos.dropna(subset=['ds'], inplace=True)
    
    # Preparar datos para Prophet
    datos_para_prophet = pd.DataFrame({
        'ds': datos['ds'], 
        'y': datos[datos.columns[1]] # asumimos que la segunda columna contiene los valores
    })
    
    # Crear y entrenar el modelo Prophet
    modelo = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
    modelo.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    modelo.fit(datos_para_prophet)
    
    # Hacer una predicción con Prophet
    futuro = modelo.make_future_dataframe(periods=pasos, freq='M')
    predicciones = modelo.predict(futuro)
    
    # Dibujar todo
    plt.figure(figsize=(12, 6))
    plt.plot(datos_para_prophet['ds'], datos_para_prophet['y'], label='Datos reales')
    plt.plot(predicciones['ds'], predicciones['yhat'], label='Predicciones con Prophet', color='red')
    plt.title(archivo)
    plt.legend()
    plt.show()

    return datos_para_prophet['y'], predicciones['yhat']

# Lista de archivos para mirar
mis_archivos = [
    "daily-total-female-births",
    "shampoo",
    "monthly-mean-temp",
    "monthly-car-sales"
]

# Hacer todo por cada archivo
for archivo in mis_archivos:
    y_real, y_pred = predice_con_prophet(archivo)
    y_pred_trunc = y_pred[:len(y_real)]

    evaluate_models(y_real, y_pred_trunc)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np

def neural_network_forecast(filename, steps=12):
    # Leer los datos
    data = pd.read_csv(filename + ".csv")
    values = data[data.columns[1]].values
    
    # Escalar los datos entre 0 y 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    values = scaler.fit_transform(values.reshape(-1, 1))

    # Preparar los datos
    X, y = [], []
    for i in range(len(values) - steps):
        X.append(values[i:i+steps, 0])
        y.append(values[i+steps, 0])
    X, y = np.array(X), np.array(y)

    # Definir el modelo
    model = Sequential()
    model.add(Dense(50, activation='relu', input_dim=steps))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Entrenar el modelo
    model.fit(X, y, epochs=200, verbose=0)

    # Hacer predicciones
    predictions = model.predict(X)
    
    # Revertir la escala
    y = scaler.inverse_transform(y.reshape(-1, 1))
    predictions = scaler.inverse_transform(predictions)

    # Visualizar
    plt.figure(figsize=(12, 6))
    plt.plot(y, label='Real')
    plt.plot(predictions, label='Predicho por NN', color='red')
    plt.legend()
    plt.title(filename)
    plt.show()
    return y, predictions


# Lista de archivos para mirar
mis_archivos = [
    "daily-total-female-births",
    "shampoo",
    "monthly-mean-temp",
    "monthly-car-sales"
]

# Hacer todo por cada archivo
for archivo in mis_archivos:
    y_true, y_pred = neural_network_forecast(archivo)
    evaluate_models(y_true, y_pred)
